In [89]:
from sklearn.datasets import load_files
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from keras.utils import np_utils, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.models import load_model
import h5py
import tensorflow as tf
import requests
from io import BytesIO
from keras.preprocessing import image
import cv2

In [90]:
# check saved model
model = load_model('../fruit recognition/fruit_classifier_model_96.h5', compile=False)
model.compile()
model

In [92]:
 # add all labels to dataset
fruit_labels = np.array([   
    "apple",
    "banana",
    "beetroot",
    "cabbage",
    "carrot",
    "cauliflower",
    "chilli pepper",
    "corn",
    "cucumber",
    "eggplant",
    "garlic",
    "ginger",
    "grapes",
    "jalepeno",
    "kiwi",
    "lemon",
    "lettuce",
    "mango",
    "onion",
    "orange",
    "pear",
    "pesa",
    "pepper",
    "pineapple",
    "pomegranate",
    "potato",
    "raddish",
    "soy beans",
    "spinach",
    "sweetcorn",
    "sweetpotato",
    "tomato",
    "turnip",
    "watermelon",
    ])  

# Konwersja etykiet na bajty (bytes)
fruit_labels_bytes = np.array([label.encode('utf-8') for label in fruit_labels])

# Zapis etykiet do pliku
with h5py.File('../fruit recognition/fruit_classifier_model_96.h5', 'a') as file:
    if 'labels' in file:
        del file['labels']
    file.create_dataset('labels', data=fruit_labels_bytes)
    model.save('fruit_classifier_model_96.h5')
# Załadowanie modelu
#model = load_model('../fruit recognition/fruit_classifier_model_97.h5')

# Zapis modelu (zawierającego etykiety)


OSError: Unable to create file (unable to truncate a file which is already open)

In [93]:
# Odczyt etykiet z pliku
with h5py.File('../fruit recognition/fruit_classifier_model_96.h5', 'r') as file:
    if 'labels' in file:
        labels_dataset = file['labels']
        fruit_labels_bytes = np.array(labels_dataset)
        fruit_labels = [label.decode('utf-8') for label in fruit_labels_bytes]
        print("Etykiety:")
        for label in fruit_labels:
            print(label)
    else:
        print("Nie znaleziono etykiet w pliku.")
    print(len(fruit_labels))

Etykiety:
apple
banana
beetroot
cabbage
carrot
cauliflower
chilli pepper
corn
cucumber
eggplant
garlic
ginger
grapes
jalepeno
kiwi
lemon
lettuce
mango
onion
orange
pear
pesa
pepper
pineapple
pomegranate
potato
raddish
soy beans
spinach
sweetcorn
sweetpotato
tomato
turnip
watermelon
34


# Test on random image from url

In [97]:
# Wczytanie modelu
model_path = 'fruit_classifier_model_96.h5'
model = load_model(model_path)


# URL obrazka do klasyfikacji
image_url = 'https://media.istockphoto.com/id/1291262112/photo/banana.jpg?s=170667a&w=0&k=20&c=BrgDCBrH3iCUHI0MXafUnkvz8lmDMelsZfKsXud1fr0=' # banana
#image_url = 'https://goodfruitguide.co.uk/wp-content/uploads/Apple-Golden-Delicious-ZA-DSC_0023-cr-sq-300x300.jpg' # apple golden

# Pobranie obrazka z podanego URL
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))


#img = load_img('../fruit recognition/images/kiwi/3.jpg')
img = img.resize((224, 224))  # Dopasowanie rozmiaru obrazka do wymagań modelu
img = img_to_array(img)
img = np.expand_dims(img, axis=0)

print('Test set shape : ',img.shape)
print('1st training image shape ',img[0].shape)
img = img.astype('float32')/255

# Klasyfikacja obrazka
predictions = model.predict(img)[0]

with h5py.File(model_path, 'r') as file:
    labels = file['labels'][:]
    fruit_names = [label.decode() for label in labels]

fruit_dict = {}
# Dopasowanie etykiet do przewidywanych wyników
for i, pred in enumerate(predictions):
    label = fruit_names[i]
    fruit_dict[label] = round(pred * 100, 2)
# Zaokrąglenie wartości poniżej 0.001 do 0.00
for label, value in fruit_dict.items():
    if value <= 0.001:
        fruit_dict[label] = 0.00
# Sortowanie etykiet według prawdopodobieństwa w kolejności malejącej
sorted_labels = sorted(fruit_dict.items(), key=lambda x: x[1], reverse=True)
# Wybieranie trzech etykiet z największym prawdopodobieństwem
top_labels = sorted_labels[:3]


predicted_class_index = np.argmax(predictions)
predicted_class = labels[predicted_class_index]
rounded_predictions = [round(pred, 1) for pred in predictions]

# Wyświetlenie wyników
print("\nPredicted class: ", predicted_class)
print("Top labels", top_labels)
print("Predictions:", [f"{label}: {pred}" for label, pred in zip(fruit_names, rounded_predictions)])



Test set shape :  (1, 224, 224, 3)
1st training image shape  (224, 224, 3)
1/1 [==============================] - 0s 174ms/step
XDDDDDDDDDD 34

Predicted class:  b'banana'
Top labels [('banana', 73.28), ('lemon', 20.77), ('orange', 5.53)]
Predictions: ['apple: 0.0', 'banana: 0.699999988079071', 'beetroot: 0.0', 'cabbage: 0.0', 'carrot: 0.0', 'cauliflower: 0.0', 'chilli pepper: 0.0', 'corn: 0.0', 'cucumber: 0.0', 'eggplant: 0.0', 'garlic: 0.0', 'ginger: 0.0', 'grapes: 0.0', 'jalepeno: 0.0', 'kiwi: 0.0', 'lemon: 0.20000000298023224', 'lettuce: 0.0', 'mango: 0.0', 'onion: 0.0', 'orange: 0.10000000149011612', 'pear: 0.0', 'pesa: 0.0', 'pepper: 0.0', 'pineapple: 0.0', 'pomegranate: 0.0', 'potato: 0.0', 'raddish: 0.0', 'soy beans: 0.0', 'spinach: 0.0', 'sweetcorn: 0.0', 'sweetpotato: 0.0', 'tomato: 0.0', 'turnip: 0.0', 'watermelon: 0.0']
